In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import time
# 内存回收
import gc
# 模型
import lightgbm as lgb
import xgboost as xgb
from xgboost import plot_importance
from tqdm import tqdm
from collections import Counter
from multiprocessing import Pool, cpu_count
# 代码运行耗时查看器
from line_profiler import LineProfiler

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.cluster import KMeans

from recall import topk_recall_association_rules_qyxs_icf, topk_recall_association_rules_ucf
from process import load_click_data, data_generate, item_cluster_feat, user_cluster_feat, \
                     matrix_word2vec_embedding, embedding_fea_pca, get_train_test_data, \
                     Pool_feature_concat, phase_submit_save, add_time_statistics, \
                     add_user_statistics, add_item_statistics, load_pool_feature, \
                     remove_pool_feature, user_feat_data, age_sex_city_data, null_xgb_predict, add_user_profile_info
from model import train_model_lgb
from metric import metrics_recall 

train_path = '../data/underexpose_train'  
test_path = '../data/underexpose_test'

have not glove
have not glove


# 内容

- 相对于V1增加了用户最喜爱session，用户不同session数
- item_n_clusters、user_n_clusters改为50
- 用户画像信息（但是重要性很低）
- lgb模型metric改为"MAP"

In [2]:
# train
phase_append = False
flag_test = False
icf_recall_num = 700
ucf_recall_num = 500
topk = 50
# 构造特征时取用户最近点击项目数
click_topn=5
# pca成分数量
pca_n_components=5
# 项目聚类数
item_n_clusters=20
# 用户聚类数
user_n_clusters=20
nrows = None 
# embedding特征与召回数据已有，直接取为True
topk_recall_read = True
embedding_feat_read = True
# 进程处理中数据划分份数或者任务数量
batchs_n = 4
# 进程中open的cpu核数
open_cpu_n = 5



# test
# phase_append = False
# flag_test = True
# recall_num = 50
# topk = 50
# nrows = 1000

# txt与img余弦相似度字典
txt_cosine_similarity_dict = np.load('../data/process/txt_cosine_similarity_dict.npy', allow_pickle=True).item()
img_cosine_similarity_dict = np.load('../data/process/img_cosine_similarity_dict.npy', allow_pickle=True).item()
type(txt_cosine_similarity_dict['42844']['67898'])

numpy.float32

In [ ]:

now_phase = 7


for phase in range(now_phase, now_phase + 1):  
    print('phase:', phase)
    """
    train_test: 训练测试点击数据
    item_hot_list: 项目热度list
    dict_label_user_item: 训练点击中用户——最后一次点击项目字典
    item_hot_dict: 项目热度字典
    time_feat：用户点击时间特征
    
    """
    click_all, test_user_set = load_click_data(phase, nrows=nrows)
    train_test, item_hot_list, dict_label_user_item, item_hot_dict, time_feat = data_generate(click_all, test_user_set)


    
    print('==================================== 召回 ====================================')
    
    # TODO: 增加其它召回
    if topk_recall_read:
        topk_recall_icf = pd.read_pickle('../data/recall/topk_recall_icf_{phase}.pkl'.format(phase=phase))
        matrix_association_rules = np.load('../data/recall/matrix_association_rules_{phase}.npy'.format(phase=phase), allow_pickle=True).item()
        topk_recall_ucf = pd.read_pickle('../data/recall/topk_recall_ucf_{phase}.pkl'.format(phase=phase))
        sim_user_corr = np.load('../data/recall/sim_user_corr_{phase}.npy'.format(phase=phase), allow_pickle=True).item()             
        
    else:
        topk_recall_icf, matrix_association_rules = topk_recall_association_rules_qyxs_icf(
                                                click_all=train_test,
                                                dict_label=dict_label_user_item, 
                                                k=icf_recall_num
                                                )
        topk_recall_icf.to_pickle('../data/recall/topk_recall_icf_{phase}.pkl'.format(phase=phase))
        np.save('../data/recall/matrix_association_rules_{phase}.npy'.format(phase=phase), matrix_association_rules)

        topk_recall_ucf, sim_user_corr = topk_recall_association_rules_ucf(click_all=train_test, 
                                                                       dict_label=dict_label_user_item, 
                                                                       k=ucf_recall_num)
        topk_recall_ucf.to_pickle('../data/recall/topk_recall_ucf_{phase}.pkl'.format(phase=phase))
        np.save('../data/recall/sim_user_corr_{phase}.npy'.format(phase=phase), sim_user_corr)

    # recall融合
    topk_recall = topk_recall_icf.merge(topk_recall_ucf, on=['user_id', 'item_similar', 'next_item_id', 'pred'], how='outer')
    
    # recall融合
    topk_recall = topk_recall_icf.merge(topk_recall_ucf, on=['user_id', 'item_similar', 'next_item_id', 'pred'], how='outer')
    
    # 缺失填补
    topk_recall = topk_recall.fillna(0)
    # 热品得分0化存在问题（这样使得数据会存在问题，不具有差异性）
    # topk_recall.loc[topk_recall.score_similar_x<0, 'score_similar_x'] = 0
    # topk_recall.loc[topk_recall.score_similar_y<0, 'score_similar_y'] = 0
    
    # 标准化
    # 因为数据取值差异较大，所有在这里只对部分取值[0,5)做了归一化
    score_similar_x=np.array(topk_recall[(topk_recall.score_similar_x>=0) & (topk_recall.score_similar_x<5)]['score_similar_x']).reshape(-1, 1)
    scaler = MinMaxScaler()
    topk_recall.loc[(topk_recall.score_similar_x>=0) & (topk_recall.score_similar_x<5), 'score_similar_x'] = scaler.fit_transform(score_similar_x).flatten()
    
    score_similar_y=np.array(topk_recall[(topk_recall.score_similar_y>=0) & (topk_recall.score_similar_y<5)]['score_similar_y']).reshape(-1, 1)
    scaler = MinMaxScaler()
    topk_recall.loc[(topk_recall.score_similar_y>=0) & (topk_recall.score_similar_y<5), 'score_similar_y'] = scaler.fit_transform(score_similar_y).flatten()
    
    # 多路召回得分融合
    topk_recall['score_similar'] = (2*topk_recall['score_similar_x'] + 1*topk_recall['score_similar_y'])/3
    topk_recall.drop(['score_similar_x', 'score_similar_y'], axis=1, inplace=True)
    
    # 每个用户得分降排，并取多路的top topn
    topn = topk_recall.groupby(['user_id'])['item_similar'].nunique().min()
    topk_recall.sort_values(['user_id', 'score_similar'], ascending=[1,0], inplace=True)
    topk_recall = topk_recall.groupby(['user_id']).head(topn)
    
        
    print('-------- 评测召回效果 -------------')
    hit_rate = metrics_recall(topk_recall=topk_recall, phase=phase, k=topn, sep=int(topn/10))
    print('召回TOP:{k}时, 命中百分比:{hit_rate}'.format(k=topn, hit_rate=hit_rate))

    
    
    print('==================================== 排序 ====================================')
    print('-------- 构建特征 ---------')
    
    dim, epochs, learning_rate = 30, 15, 0.1 # 由于内存问题，size改回30
    if embedding_feat_read:
        print('-------- sku1 sku2 sku3 sku4 sku5 user ----------')
        dict_embedding_all_ui_item = np.load('../data/embedding/dict_embedding_all_ui_item_{phase}.npy'.format(phase=phase), 
                                             allow_pickle=True).item()
        
        print('------- user1 user2 user3 user4 user5 sku -------')
        dict_embedding_all_ui_user = np.load('../data/embedding/dict_embedding_all_ui_user_{phase}.npy'.format(phase=phase), 
                                             allow_pickle=True).item()
        
        print('------- item1 item2 item3 item4 item5 -------')
        dict_embedding_item_only = np.load('../data/embedding/dict_embedding_item_only_{phase}.npy'.format(phase=phase), 
                                           allow_pickle=True).item()
        
        print('------- user1 user2 user3 user4 user5 -------')
        dict_embedding_user_only = np.load('../data/embedding/dict_embedding_user_only_{phase}.npy'.format(phase=phase), 
                                           allow_pickle=True).item()

    else:
        print('-------- sku1 sku2 sku3 sku4 sku5 user ----------')
        dict_embedding_all_ui_item = matrix_word2vec_embedding(
                                                                click_all=train_test,
                                                                flag='item',
                                                                mode='all',
                                                                dim=dim,
                                                                epochs=epochs,
                                                                learning_rate=learning_rate
                                                                )
        np.save('../data/embedding/dict_embedding_all_ui_item_{phase}.npy'.format(phase=phase),
               dict_embedding_all_ui_item)


        print('------- user1 user2 user3 user4 user5 sku -------')
        dict_embedding_all_ui_user = matrix_word2vec_embedding(
                                                                click_all=train_test,
                                                                flag='user',
                                                                mode='all',
                                                                dim=dim,
                                                                epochs=epochs,
                                                                learning_rate=learning_rate
                                                                )
        np.save('../data/embedding/dict_embedding_all_ui_user_{phase}.npy'.format(phase=phase),
               dict_embedding_all_ui_user)


        print('------- item1 item2 item3 item4 item5 -------')
        dict_embedding_item_only = matrix_word2vec_embedding(
                                                                click_all=train_test,
                                                                flag='item',
                                                                mode='only',
                                                                dim=dim,
                                                                epochs=epochs,
                                                                learning_rate=learning_rate
                                                                )
        np.save('../data/embedding/dict_embedding_item_only_{phase}.npy'.format(phase=phase),
               dict_embedding_item_only)


        print('------- user1 user2 user3 user4 user5 -------')
        dict_embedding_user_only = matrix_word2vec_embedding(
                                                                click_all=train_test,
                                                                flag='user',
                                                                mode='only',
                                                                dim=dim,
                                                                epochs=epochs,
                                                                learning_rate=learning_rate
                                                                )
        np.save('../data/embedding/dict_embedding_user_only_{phase}.npy'.format(phase=phase),
               dict_embedding_user_only)

        
    
    print('------- Interactive Embedding Feature PCA -----------')

    # dict_embedding_all_ui_item PCA
    dict_embedding = dict_embedding_all_ui_item.copy()
    dict_embedding_all_ui_item = {}
    for i in ['user', 'item']:
        tmp = pd.DataFrame(dict_embedding[i])
        pca_ = embedding_fea_pca(tmp.T, n_components=pca_n_components) # 解释
        tmp_pca = pd.DataFrame(pca_.T)
        tmp_pca.columns = tmp.columns
        dict_embedding_all_ui_item[i] = dict(tmp_pca)
    
    
    # dict_embedding_all_ui_user PCA
    dict_embedding = dict_embedding_all_ui_user.copy()
    dict_embedding_all_ui_user = {}
    for i in ['user', 'item']:
        tmp = pd.DataFrame(dict_embedding[i])
        pca_ = embedding_fea_pca(tmp.T, n_components=pca_n_components) # 解释
        tmp_pca = pd.DataFrame(pca_.T)
        tmp_pca.columns = tmp.columns
        dict_embedding_all_ui_user[i] = dict(tmp_pca)
    print('dict_embedding_all_ui_user是否降维成功........', len(dict_embedding_all_ui_user['user']['1']))    
    del tmp, tmp_pca

    
    
    print('------- user or item Embedding Feature Cluster -----------')

    # dict_embedding_item_only聚类
    item = pd.DataFrame(dict_embedding_item_only['item'])
    km_cluster = KMeans(n_clusters=item_n_clusters, max_iter=300, n_init=40, \
                        init='k-means++') #, n_jobs=-1)
    # 返回类索引
    result = km_cluster.fit_predict(item.T)
    item_cluster = pd.DataFrame({'item_id': item.columns, 'item_cluster': np.int16(result)})
    del item, km_cluster, result, dict_embedding_item_only
    
    # dict_embedding_user_only聚类
    user = pd.DataFrame(dict_embedding_user_only['user'])
    km_cluster = KMeans(n_clusters=user_n_clusters, max_iter=300, n_init=40, \
                        init='k-means++') #, n_jobs=-1)
    # 返回类索引
    result = km_cluster.fit_predict(user.T)
    user_cluster = pd.DataFrame({'user_id': user.columns, 'user_cluster': np.int16(result)})
    del user, km_cluster, result
    
    
    # 加载聚类的统计特征
    user_cluster_feat_ = user_cluster_feat(click_all, user_cluster)
    item_cluster_feat_ = item_cluster_feat(click_all, item_cluster)

    del click_all, item_cluster
    
    
    print('------- 特征加工 -----------')

    print('..............topk_recall shape:', topk_recall.shape)

    
#     feature_all = get_train_test_data(topk_recall,
#                   matrix_association_rules, 
#                   time_feat,
#                   user_cluster_feat_,
#                   item_cluster_feat_,
#                   item_hot_dict,
#                   txt_cosine_similarity_dict,
#                   img_cosine_similarity_dict,
#                   dict_embedding_all_ui_item,
#                   dict_embedding_all_ui_user,
#                   flag_test,
#                   click_topn)


#     lprofiler = LineProfiler(get_train_test_data)
#     lprofiler.run('get_train_test_data( topk_recall[:80000], matrix_association_rules, time_feat, user_cluster_feat_, item_cluster_feat_, item_hot_dict, txt_cosine_similarity_dict, img_cosine_similarity_dict, dict_embedding_all_ui_item, dict_embedding_all_ui_user, flag_test=False, click_topn=click_topn)')
#     lprofiler.print_stats()

    
    # 内存回收
    gc.collect()
    # 多进程构建特征
    # 函数get_train_test_data内部已保存结果
    if __name__=='__main__':
        cpu_num = cpu_count()
        
        pool = Pool(open_cpu_n+1) # 创建一个多个进程的进程池
        # 将topk_recall分batchs_n批次，并不是越大越好，启动进程也是需时间的
        batch_size = topk_recall.shape[0]//(batchs_n)
        
        for batch in range(batchs_n):
            start_index = batch * batch_size
            if batch==cpu_num:
                end_index = topk_recall.shape[0]
            else:
                end_index = (1+batch) * batch_size
            p = pool.apply_async(func=get_train_test_data, 
                                                 args=(topk_recall[start_index:end_index],
                                                       matrix_association_rules, 
                                                       time_feat,
                                                       user_cluster_feat_,
                                                       item_cluster_feat_,
                                                       item_hot_dict,
                                                       txt_cosine_similarity_dict,
                                                       img_cosine_similarity_dict,
                                                       dict_embedding_all_ui_item,
                                                       dict_embedding_all_ui_user,
                                                       phase,
                                                       batch,
                                                       flag_test,
                                                       click_topn))            
        pool.close()
        pool.join() 
        
        """
        遍历result列表，取出子进程对象，访问get()方法，获取返回值。（此时所有子进程已执行完毕）
        result[0].get()
        """
        print('进程结束') 
     
    
    
#     # 释放内存
# #     del topk_recall, matrix_association_rules, txt_cosine_similarity_dict, img_cosine_similarity_dict, dict_embedding_all_ui_item, dict_embedding_all_ui_user, dict_embedding_item_only, dict_embedding_user_only
      
    print('分片特征数据拼接')
    # feature_all = Pool_feature_concat(feature_part)
    # 加载暂存的特征数据
    feature_all = load_pool_feature(phase=phase, batchs_n=batchs_n)
    # 删除暂存的特征数据
    remove_pool_feature(phase, batchs_n)
    
    
    # 用户画像特征缺失预测
    user_feat = user_feat_data(dict_embedding_user_only, user_cluster, time_feat)
    for i, col in enumerate(['age', 'sex', 'city']):
        X_, y_, X_test, data_ = age_sex_city_data(user_feat, col=col)
        tmp = null_xgb_predict(X_, y_, X_test, data_, col=col)
        if i==0:
            user_profile_feat = tmp
        else:
            user_profile_feat = user_profile_feat.merge(tmp, on=['user_id'], how='left')    

    # 增加用户画像特征、时间统计特征、用户聚类统计特征、项目聚类统计特征 
    feature_all = add_user_profile_info(feature_all, user_profile_feat)
    feature_all = add_time_statistics(feature_all, time_feat)
    feature_all = add_user_statistics(feature_all, user_cluster_feat_)
    feature_all = add_item_statistics(feature_all, item_cluster_feat_)
    print('feature_all的列: ', feature_all.columns)
    
    
#     # 数据量太大了，保存占用空间
#     print('特征数据保存')
#     feature_all.to_pickle("../data/process/feature_all_phase{phase}_0002.pkl".format(phase=phase))
    
    # 内存回收
    gc.collect()
    ############################## 整体数据特征 ############################## 
    print('--------------------------- 特征数据 ---------------------')
    len_f = len(feature_all)
    len_train = len(feature_all[feature_all['train_flag']=='train'])
    len_test = len(feature_all[feature_all['train_flag']=='test'])
    len_train_1 = len(feature_all[(feature_all['train_flag']=='train') & (feature_all['label']== 1)]) 
    print('所有数据条数', len_f)
    print('训练数据 : ', len_train)
    print('训练数据 label 1 : ', len_train_1)
    print('训练数据 1 / 0 rate : ', len_train_1 * 1.0 / len_f)
    print('测试数据 : ' , len_test)
    print('flag : ', set(feature_all['train_flag']))

    
    ############################## 训练模型 ############################## 
    print('--------------------------- 训练模型 ---------------------')
    submit = train_model_lgb(feature_all, recall_rate=hit_rate, hot_list=item_hot_list, valid=0.2, topk=50, 
                             num_boost_round=1500, early_stopping_rounds=500)

    
    # submit = train_model_rf(feature_all, recall_rate=hit_rate, hot_list=item_hot_list, valid=0.2, topk=50)
    

    print('--------------------------- 保存预测文件 ---------------------')
    phase_submit_save(submit, phase, 50)


phase: 7
================================== 加载click数据 ==================================
================================== 生成中间数据 ==================================
==================================== 召回 ====================================
-------- 评测召回效果 -------------
-------- 召回效果 -------------
--------:phase:  7  -------------
phase:  7  top_ 0  :  hit_num :  303 hit_rate :  0.016829593423683627  data_num :  18004

phase:  7  top_ 70  :  hit_num :  2324 hit_rate :  0.1290824261275272  data_num :  18004

phase:  7  top_ 140  :  hit_num :  2897 hit_rate :  0.16090868695845367  data_num :  18004

phase:  7  top_ 210  :  hit_num :  3211 hit_rate :  0.1783492557209509  data_num :  18004

phase:  7  top_ 280  :  hit_num :  3406 hit_rate :  0.1891801821817374  data_num :  18004

phase:  7  top_ 350  :  hit_num :  3575 hit_rate :  0.19856698511441903  data_num :  18004

phase:  7  top_ 420  :  hit_num :  3711 hit_rate :  0.20612086203065985  data_num :  18004

phase:  7  top_ 490  :  hit

14358it [00:27, 519.69it/s]

------- 构建样本 -----------


16098it [00:31, 519.35it/s]

............... train len= 723275
............... test  len= 310100


...

49390it [01:36, 513.58it/s]

............... train len= 752500
............... test  len= 310100


49494it [01:37, 514.47it/s]

----------- 加入特征 train -----------


420786it [14:38, 511.66it/s]